In [42]:
from time import time
import logging
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.tree import DecisionTreeClassifier

In [3]:
#  Images are single-channel with pixel intensity represented by a float value from 0 to 255,
#  the single-channel intensity is the mean value of the original RGB channel intensities.

# X_train.npy: training data numpy array.
# Each row in the array corresponds to an image unrolled to a vector (50 x 37 = 1850 dimension)
X_train = np.load('X_train.npy')
# y_train.npy: labels (0-6) of each data corresponding to the image in the same row in X_train.npy
y_train = np.load('y_train.npy')

# Test data without label
X_test = np.load('X_test.npy')

In [50]:
# Generate testResults.csv
# Run this with y_pred

def saveTestResults(y_pred, filename='testResults.csv'):
    fo = open(filename, 'w')
    fo.write("ImageId,PredictedClass\n")
    for i in range(len(y_pred)):
        if i == len(y_pred) - 1:
            fo.write(str(i) + "," + str(y_pred[i]))
        else:
            fo.write(str(i) + "," + str(y_pred[i]) + "\n")
    fo.close()

In [35]:
# Reduce dimensionality with PCA (i.e. eigenfaces)
all_data = []
for i in range(len(y_train)):
    is_same = False
    for j in range(len(all_data)):
        if y_train[i] == all_data[j]:
            is_same = True
            break
    if is_same != True:
        all_data.append(y_train[i])


# Display progress logs on stdout
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')

n_classes = len(all_data)
n_samples = len(X_train)
n_features = len(X_train[0])

print("Total dataset size:")
print("n_samples: %d" % n_samples)
print("n_features: %d" % n_features)
print("n_classes: %d" % n_classes)

n_components = 150
h = 1
w = 1850
print("Extracting the top %d eigenfaces from %d faces"
      % (n_components, X_train.shape[0]))
t0 = time()
pca = PCA(n_components=n_components, svd_solver='randomized',
          whiten=True).fit(X_train)
print("done in %0.3fs" % (time() - t0))

eigenfaces = pca.components_.reshape((n_components, h, w))
print("Projecting the input data on the eigenfaces orthonormal basis")

t0 = time()
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

print("done in %0.3fs" % (time() - t0))

print(X_train_pca.shape)

# Split data (75% train, 25% test)
X_test_pca_split, ZX_test_pca_split, y_test_pca_split, Zy_test_pca_split = train_test_split(X_train_pca, y_train, test_size=0.25, random_state=42)

Total dataset size:
n_samples: 966
n_features: 1850
n_classes: 7
Extracting the top 150 eigenfaces from 966 faces
done in 0.182s
Projecting the input data on the eigenfaces orthonormal basis
done in 0.023s
(966, 150)


In [51]:
# GridSearchCV with SVC
print("Fitting the classifier to the training set")
t0 = time()

# Runs 1 vs 1 NOT 1 vs REST.
param_grid = {'C': [1, 3, 5, 1e1, 5e1, 1e2, 1e3],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], }
clf = GridSearchCV(estimator=SVC(kernel='rbf', class_weight='balanced'), param_grid=param_grid, cv=10)
clf = clf.fit(X_train_pca, y_train)
print("done in %0.3fs" % (time() - t0))
print("Best estimator found by grid search:")
print(clf.best_estimator_)

print("Error in")
internal_pred = clf.predict(ZX_test_pca_split)
print(classification_report(Zy_test_pca_split, internal_pred))

print("Predicting people's names on the test set")
t0 = time()
y_pred = clf.predict(X_test_pca)
print("done in %0.3fs" % (time() - t0))

print(y_pred)
saveTestResults(y_pred, 'testResults_SVCwithBestEstimator.csv')

Fitting the classifier to the training set
done in 230.129s
Best estimator found by grid search:
SVC(C=1, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.005, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
Error in
             precision    recall  f1-score   support

          0       1.00      1.00      1.00        13
          1       1.00      1.00      1.00        40
          2       1.00      1.00      1.00        26
          3       1.00      1.00      1.00        92
          4       1.00      1.00      1.00        33
          5       1.00      1.00      1.00        11
          6       1.00      1.00      1.00        27

avg / total       1.00      1.00      1.00       242

Predicting people's names on the test set
done in 0.060s
[3 4 6 1 3 2 2 6 1 1 1 3 3 3 4 0 1 1 3 1 3 0 0 2 3 0 4 1 4 3 2 6 3 1 1 6 3
 6 3 6 3 6 3 1 3 3 4 2 3 3 3 3 3 3 4 2 1 3 4 0 6 4 2 3 

In [52]:
# DT with Adaboost
print("Fitting the classifier to the training set")
t0 = time()

print("Performing with Adaboost")
clf = AdaBoostClassifier(DecisionTreeClassifier(class_weight='balanced'), n_estimators=300)
clf = clf.fit(X_test_pca_split, y_test_pca_split)

# scores = cross_val_score(clf, X_train_pca, y_train)
# print("Mean scores:", scores.mean())

print("Error in")
internal_pred = clf.predict(ZX_test_pca_split)
print(classification_report(Zy_test_pca_split, internal_pred))

print("Predicting people's names on the test set")
t0 = time()
y_pred = clf.predict(X_test_pca)
print("done in %0.3fs" % (time() - t0))

print(y_pred)
saveTestResults(y_pred, 'testResults_DTwithAdaboost.csv')

Fitting the classifier to the training set
Performing with Adaboost
Error in
             precision    recall  f1-score   support

          0       0.20      0.23      0.21        13
          1       0.37      0.53      0.43        40
          2       0.30      0.27      0.29        26
          3       0.43      0.42      0.43        92
          4       0.13      0.06      0.08        33
          5       0.17      0.18      0.17        11
          6       0.30      0.33      0.32        27

avg / total       0.33      0.34      0.33       242

Predicting people's names on the test set
done in 0.001s
[3 3 3 0 6 3 2 5 6 4 6 6 3 5 5 2 3 1 2 3 0 1 3 3 3 1 1 1 5 6 3 3 1 6 3 3 3
 6 3 3 3 3 2 1 1 3 1 3 6 2 3 3 3 4 1 3 3 1 3 0 6 3 2 3 4 3 5 3 2 5 3 1 4 6
 3 2 2 5 5 3 5 3 6 3 6 0 2 6 2 3 2 1 1 3 6 2 3 2 3 6 1 4 5 3 1 3 4 4 2 5 3
 3 3 3 3 2 6 1 2 3 1 3 2 3 6 3 5 4 3 4 3 3 2 1 1 1 0 0 6 3 1 3 3 2 6 3 5 3
 3 3 3 4 5 3 6 3 1 6 3 4 1 6 3 3 3 2 6 2 1 3 6 2 3 3 1 1 3 3 2 1 3 3 1 3 1
 6 2 1 1 4 

In [53]:
# Linear SVC with Adaboost
print("Fitting the classifier to the training set")
t0 = time()

print("Performing with Adaboost")
clf = AdaBoostClassifier(SVC(kernel='linear', class_weight='balanced', C=1, gamma=0.005, probability=True), n_estimators=300, algorithm='SAMME')
clf = clf.fit(X_test_pca_split, y_test_pca_split)

print("Error in")
internal_pred = clf.predict(ZX_test_pca_split)
print(classification_report(Zy_test_pca_split, internal_pred))

print("Predicting people's names on the test set")
t0 = time()
y_pred = clf.predict(X_test_pca)
print("done in %0.3fs" % (time() - t0))

print(y_pred)
saveTestResults(y_pred, 'testResults_LinearSVCwithAdaboost.csv')

Fitting the classifier to the training set
Performing with Adaboost
Error in
             precision    recall  f1-score   support

          0       1.00      0.46      0.63        13
          1       0.92      0.85      0.88        40
          2       0.86      0.46      0.60        26
          3       0.66      0.91      0.76        92
          4       0.80      0.61      0.69        33
          5       0.90      0.82      0.86        11
          6       0.77      0.63      0.69        27

avg / total       0.78      0.75      0.75       242

Predicting people's names on the test set
done in 14.468s
[3 3 6 1 3 2 0 3 0 1 1 3 3 3 3 0 1 1 4 1 3 0 0 0 3 0 3 1 6 3 2 6 3 2 1 6 3
 6 5 6 3 3 3 1 3 3 2 0 2 4 3 3 3 3 4 3 1 0 4 0 6 4 2 0 4 3 1 3 3 3 3 3 3 2
 3 3 2 3 3 3 3 3 1 3 6 0 3 1 2 1 0 4 2 3 6 3 3 5 1 3 3 6 3 3 4 3 3 6 3 4 3
 3 0 5 3 1 6 5 3 3 2 0 5 3 6 3 5 1 2 3 2 3 3 3 3 3 1 2 3 3 1 6 1 3 3 1 1 2
 3 3 3 4 1 1 6 3 5 3 3 3 1 3 3 0 5 4 6 2 3 1 3 2 3 3 3 1 5 3 3 3 0 3 1 0 0
 6 1 1 3 5

In [20]:
# print(classification_report(y_test, y_pred))
# print(confusion_matrix(y_test, y_pred, labels=range(n_classes)))
#
# def plot_gallery(images, titles, h, w, n_row=3, n_col=4):
#     """Helper function to plot a gallery of portraits"""
#     plt.figure(figsize=(1.8 * n_col, 2.4 * n_row))
#     plt.subplots_adjust(bottom=0, left=.01, right=.99, top=.90, hspace=.35)
#     for i in range(n_row * n_col):
#         plt.subplot(n_row, n_col, i + 1)
#         plt.imshow(images[i].reshape((h, w)), cmap=plt.cm.gray)
#         plt.title(titles[i], size=12)
#         plt.xticks(())
#         plt.yticks(())
#
#
# # plot the result of the prediction on a portion of the test set
#
# # plot the gallery of the most significative eigenfaces
#
# eigenface_titles = ["eigenface %d" % i for i in range(eigenfaces.shape[0])]
# plot_gallery(eigenfaces, eigenface_titles, h, w)
#
# plt.show()